In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import random as rd

In [2]:
# import some data to play with
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

# Convert the data to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train)
y_train_tensor = F.one_hot(y_train_tensor, num_classes=3).float()
X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val)
y_val_tensor = F.one_hot(y_val_tensor, num_classes=3).float()

In [4]:
(X_train_tensor.shape[0], X_val_tensor.shape[0])

(135, 15)

In [5]:
# Define the MLP architecture
class Iris_MLP(nn.Module):
    def __init__(self):
        super(Iris_MLP, self).__init__()
        self.ly1 = nn.Linear(4, 20)
        self.ly2 = nn.Linear(20, 20)
        self.final = nn.Linear(20, 3)
        
        self.sig = nn.Sigmoid()
        
        
    def forward(self, x):
        out = self.ly1(x)
        out = self.sig(out)
        out = self.ly2(out)
        out = self.sig(out)
        out = self.final(out)
        return out

In [6]:
# Create an instance of the MLP model
model = Iris_MLP()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.03)

In [7]:
model.eval()
y_pred = model(X_val_tensor)
before_train = criterion(y_pred.squeeze(), y_val_tensor)
print('Test loss before training' , before_train.item())

Test loss before training 0.31159159541130066


In [8]:
# Train the model
model.train()
epoch = 5000

for epoch in range(epoch):
    # Forward pass
    y_pred = model(X_train_tensor)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train_tensor)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    
    # Zero grad
    optimizer.zero_grad()

    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.3195977210998535
Epoch 1: train loss: 0.2611084282398224
Epoch 2: train loss: 0.26518288254737854
Epoch 3: train loss: 0.25922852754592896
Epoch 4: train loss: 0.23850348591804504
Epoch 5: train loss: 0.2181243747472763
Epoch 6: train loss: 0.20907799899578094
Epoch 7: train loss: 0.20991556346416473
Epoch 8: train loss: 0.2100074291229248
Epoch 9: train loss: 0.2009638398885727
Epoch 10: train loss: 0.1847846955060959
Epoch 11: train loss: 0.17052096128463745
Epoch 12: train loss: 0.16504503786563873
Epoch 13: train loss: 0.16517874598503113
Epoch 14: train loss: 0.16146771609783173
Epoch 15: train loss: 0.1501287966966629
Epoch 16: train loss: 0.13679829239845276
Epoch 17: train loss: 0.12898197770118713
Epoch 18: train loss: 0.1273760199546814
Epoch 19: train loss: 0.12596315145492554
Epoch 20: train loss: 0.12112215161323547
Epoch 21: train loss: 0.11643597483634949
Epoch 22: train loss: 0.11668068915605545
Epoch 23: train loss: 0.11977354437112808
Epoch 24: 

In [9]:
model.eval()
y_pred = model(X_val_tensor)
after_train = criterion(y_pred.squeeze(), y_val_tensor)
print('Test loss after training' , after_train.item())

Test loss after training 0.0017063660779967904


# Run a test using predefined values

In [10]:
X_input_indx = rd.randint(0, len(X)-1)

(X[X_input_indx], iris.target_names[y[X_input_indx]])

(array([6.7, 3.3, 5.7, 2.5]), 'virginica')

In [11]:
model.eval()
output = (model(torch.from_numpy(X[X_input_indx]).float()) ** 2).sqrt()

((iris.target_names[0], (output / output.sum())[0].item() * 100),
 (iris.target_names[1], (output / output.sum())[1].item() * 100),
 (iris.target_names[2], (output / output.sum())[2].item() * 100))

(('setosa', 0.09996476583182812),
 ('versicolor', 0.27641793712973595),
 ('virginica', 99.62361454963684))